### 임포트, 초기설정

In [ ]:
#import packages

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys  # Keys 모듈 import
from selenium.webdriver.support.ui import Select
import time
import datetime
import urllib.request
from PyPDF2 import PdfReader

import pandas as pd
import os
import math
import re
from collections import OrderedDict
#이미지
import requests
from PIL import Image
from io import BytesIO
import pytesseract
import subprocess
import sys

import csv
from datetime import datetime
from selenium.common.exceptions import StaleElementReferenceException

import requests
from bs4 import BeautifulSoup


### 임포트, 초기설정

In [ ]:
# 고정 변수
start_url = 'https://spib.wooribank.com/pib/Dream?withyou=CMLGN0001'   #우리은행예금페이지
fund_login = 'https://spot.wooribank.com/pot/Dream?withyou=OWFDM0003'
fund_url_1 = 'https://spot.wooribank.com/pot/Dream?withyou=OWFDM0006'   #상세펀드페이지
fund_url_2 = 'https://spot.wooribank.com/pot/Dream?withyou=OWFDM0019'   #연금펀드페이지
user_name = 'SYSTEM2'
file_path = 'C:/Users/WAI/KB_img_tmp'

#폴더 생성

# try:
#     if not os.path.exists(file_path):
#         os.mkdir(file_path)
# except:
#     print("error : 폴더를 만들수없음")

# 이미지를 저장할 하위 폴더 생성
protect_img = 'protect_fund_img'
os.makedirs(protect_img, exist_ok=True)  # 폴더가 이미 존재하면 무시하도록 설정


# 이미지를 저장할 하위 폴더 생성
product_img = 'product_fund_img'
os.makedirs(product_img, exist_ok=True)  # 폴더가 이미 존재하면 무시하도록 설정

In [ ]:
chromeOptions = webdriver.ChromeOptions()
# chromeOptions.add_argument("--headless")
prefs = {"download.default_directory" : r"C:\Users\Administrator/KB_img_tmp\\"}#원하는 위치에 다운로드
chromeOptions.add_experimental_option("prefs",prefs)
# chromedriver = "chromedriver.exe"#상대경로
service = Service(executable_path='./chromedriver.exe')



driver = webdriver.Chrome(service=service, options=chromeOptions)
#driver.get(start_url) #전역변수

# 크롬 창을 창 모드로 최대화합니다
#driver.set_window_position(0, 0)
#driver.set_window_size(1800, 1024)  # 원하는 창 크기로 설정

### 크롤링

In [ ]:
def WB_IDPW():
    ######################################################################
    ######################################################################
    ##########아이디 비번 입력 프로그램 실행################################
    ######################################################################
    ######################################################################
    import subprocess

    # 실행할 외부 프로그램의 경로 및 옵션 설정
    executable_name = "우리은행로그인.exe"

    # 외부 프로그램 실행
    process = subprocess.Popen(executable_name, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # 외부 프로그램의 출력 받기
    stdout, stderr = process.communicate()

    # 외부 프로그램의 표준 출력 및 표준 에러 출력
    print("표준 출력:")
    print(stdout)

    print("\n표준 에러 출력:")
    print(stderr)

    # 외부 프로그램의 종료 코드 확인
    return_code = process.returncode
    print(f"종료 코드: {return_code}")

In [ ]:
############## 알림창끄기 함수
def Alert():
    try:
        alert = driver.switch_to.alert
        alert.accept() 
    except:
        pass 

### PDF 부분

In [ ]:
###### pdf함수 1 pdf주소 어펜드
def gain_pdf_url(driver, classname):
    ##########pdf 어펜드
    try:
        product_pdf_name = []
        product_pdf_list = []
        pdf_box = driver.find_element(By.CLASS_NAME,classname)
        pdf_lists = pdf_box.find_elements(By.TAG_NAME,'li')
    except:
        product_pdf_name.append('')
        product_pdf_list.append('')
        return product_pdf_name, product_pdf_list

        
    for pdf_list in pdf_lists:
        pdf_url = pdf_list.find_element(By.TAG_NAME, 'a').get_attribute('href')
        product_pdf_name.append(pdf_list.text)
        product_pdf_list.append(pdf_url)
    return product_pdf_name, product_pdf_list

In [ ]:
##### pdf함수 2 예보 문구 찾기
def find_guddoc_from_pdf(text):
    find_text = "예금자보호\n여부해당"#예금, 외화
    if text.find(find_text) == -1:
        find_text = "예금자보호안내"#보험(안내장)
    if text.find(find_text) == -1:
        print("#######################",  "ERROR   ","#######################")

    cut_text = text[text.find(find_text) + len(find_text):].replace("\n", "")
    yebo_string = cut_text[:cut_text.find(".")].strip().replace("   ", " ").replace("  ", " ")
    
    return yebo_string

In [ ]:
# pdf_box = driver.find_element(By.CLASS_NAME,'info-down')
# pdf_lists = pdf_box.find_elements(By.TAG_NAME,'li')
# pdf_lists

In [ ]:
# gain_pdf_url(driver, 'info-down')

In [ ]:
#### 
######pdf 함수3 내용 크롤링

def cr_pdf(product_pdf_name, pdf_list):
    if pdf_list != '':
        print('cr_pdf 출력 : PDF 있음 크롤링 함수 시작')
        pdf_text = []
        pdf_protect_text = []
        ##pdf 크롤링
        for i in range(len(pdf_list)):
            protect_word = []
            #print('cr_pdf 출력 : ' + str(i) + '번째 PDF 크롤링 중')
            pdf_name = product_pdf_name[i]
            pdf_response = requests.post(pdf_list[i])
            file = open(f"test.pdf", "wb")
            #file = open(pdf_filename, "wb")
            file.write(pdf_response.content)
            file.close()        
            time.sleep(1)
            
            reader = PdfReader("test.pdf")

            n = len(reader.pages) # get all pages in one
            text='' # saving array
            text += pdf_name
            for j in range(n): # loop
                contents = reader.pages[j].extract_text()
                text += contents
                
            # '\n'을 공백으로 대체하여 제거
            text = text.replace('\n', '')
            
            sentences = re.split(r'(?<=[.!?])', text)
            for k in range(len(sentences)):
                if re.search(r'(?:예금자\s*보호|예금자보호)', sentences[k]):
                    #protect_word.append(pdf_name + str(k) + ' : ' + sentences[k])
                    protect_word.append(sentences[k])
                else:
                    #protect_word.append(pdf_name + str(k) + ' : 예금자 보호 문구가 없음')
                    #print('cr_pdf 출력 : 예금자 보호문구가 없음')
                    continue


            #print(protect_word[:20])
            protect_words = ''        
            if protect_word:  # protect_word가 비어 있지 않은 경우에만 조인을 수행합니다.
                protect_words = ', '.join(protect_word)
                #print(protect_words)
            # else:
            #     print('protect_word가 비어 있습니다.')
            # protect_words = '; '.join(protect_word)
            #print(protect_wordss[:50])


            pdf_text.append(text)
            pdf_protect_text.append(protect_words)


        combined_pdf_text = '\n'.join(pdf_text)
        combined_pdf_protect_text = ', '.join(pdf_protect_text)
        print('cr_pdf 출력 :  ' + combined_pdf_protect_text[:50])
        print('cr_pdf 출력 : PDF 크롤링 완료')
        return combined_pdf_text, combined_pdf_protect_text
            #protect_text = find_guddoc_from_pdf(text)
    else:
        print('cr_pdf 출력 : PDF 없음 확인 바람')
        pdf_text.append('cr_pdf 출력 : PDF 없음 확인 바람')
        #pdf_protect_text.append('PDF 없음 확인 바람')
        pdf_text = ''
        pdf_protect_text = ''
        return combined_pdf_text, combined_pdf_protect_text




In [ ]:
#### 
######pdf 함수3 내용 크롤링

def cr_pdf(product_pdf_name, pdf_list):
    if not pdf_list:
        print('cr_pdf 출력 : PDF 없음 확인 바람')
        return '', ''

    print('cr_pdf 출력 : PDF 있음 크롤링 함수 시작')
    pdf_text = []
    pdf_protect_text = []
    ##pdf 크롤링
    for i in range(len(pdf_list)):
        protect_word = []
        #print('cr_pdf 출력 : ' + str(i) + '번째 PDF 크롤링 중')
        pdf_name = product_pdf_name[i]
        pdf_response = requests.post(pdf_list[i])
        file = open(f"test.pdf", "wb")
        #file = open(pdf_filename, "wb")
        file.write(pdf_response.content)
        file.close()        
        time.sleep(1)
        
        reader = PdfReader("test.pdf")

        n = len(reader.pages) # get all pages in one
        text='' # saving array
        text += pdf_name
        for j in range(n): # loop
            contents = reader.pages[j].extract_text()
            text += contents
            
        # '\n'을 공백으로 대체하여 제거
        text = text.replace('\n', '')
        
        sentences = re.split(r'(?<=[.!?])', text)
        for k in range(len(sentences)):
            if re.search(r'(?:예금자\s*보호|예금자보호)', sentences[k]):
                #protect_word.append(pdf_name + str(k) + ' : ' + sentences[k])
                protect_word.append(sentences[k])
            else:
                #protect_word.append(pdf_name + str(k) + ' : 예금자 보호 문구가 없음')
                #print('cr_pdf 출력 : 예금자 보호문구가 없음')
                continue


        #print(protect_word[:20])
        protect_words = ''        
        if protect_word:  # protect_word가 비어 있지 않은 경우에만 조인을 수행합니다.
            protect_words = ', '.join(protect_word)
            #print(protect_words)
        # else:
        #     print('protect_word가 비어 있습니다.')
        # protect_words = '; '.join(protect_word)
        #print(protect_wordss[:50])


        pdf_text.append(text)
        if protect_word:  # protect_word가 비어 있지 않은 경우에만 조인을 수행합니다.
            protect_words = ', '.join(protect_word)
            pdf_protect_text.append(protect_words)


    combined_pdf_text = '\n'.join(pdf_text)
    combined_pdf_protect_text = ', '.join(pdf_protect_text)
    print(combined_pdf_protect_text[:20])
    print('cr_pdf 출력 : PDF 크롤링 완료')
    
    return combined_pdf_text, combined_pdf_protect_text
        #protect_text = find_guddoc_from_pdf(text)
    # else:
    #     print('cr_pdf 출력 : PDF 없음 확인 바람')
    #     pdf_text.append('cr_pdf 출력 : PDF 없음 확인 바람')
    #     #pdf_protect_text.append('PDF 없음 확인 바람')
    #     pdf_text = ''
    #     pdf_protect_text = ''
    #     return combined_pdf_text, combined_pdf_protect_text




In [ ]:
##pdf 크롤링하기
# pdf_list = gain_pdf_url(driver, 'info-down')
# pdf_text = cr_pdf(pdf_list)

In [ ]:
# df_fund_2 = wb_login_cr(fund_url_2, '연금펀드.csv', 1)
# df_fund_2.to_csv('연금펀드2.csv', sep=',', encoding = 'utf-8-sig',index=None)

### 리스트페이지변경

In [ ]:
############## 리스트바꾸고 페이지까지 체크하는 함수
def ChagelistandCheck(listnum, max_attemps, page_now, findbtnclick):
    findbtnclick = int(findbtnclick)
    print('ChagelistandCheck 출력 : 리스트' + str(listnum) + '개로 변경 완료')
    #print(findbtnclick)
    max_attempts = int(max_attemps)
    for attempt in range(max_attempts):
        #print(attempt)
        #print('첫번째try전')
        #try:
        ##time.sleep(1)
        select_element = driver.find_element(By.ID, "RowCnt")
        select = Select(select_element)
        select.select_by_value(str(listnum))
        #test time.sleep(1)
        #print('스크롤내리기전')
        for k in range(5):
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(0.1)
        if findbtnclick == 1 :
            driver.find_element(By.CLASS_NAME, 'btn-search').click()
            #print('리스트개수 변경완료1')
            time.sleep(1)   #페이지변경시 에러나면 여기 늘리기
        else:
            #print('리스트개수 변경완료2')
            pass
        #print('111')
        table = driver.find_element(By.TAG_NAME, 'tbody') # 상품 테이블 찾기
        product_list = table.find_elements(By.XPATH, "//*[starts-with(@onclick, '$.nextPage(1,')]")  # 현재 페이지의 상품 개수 찾기
        
        check_product_list = 0
        while check_product_list < 10:
            if product_list == '':
                time.sleep(0.2)
                check_product_list += 1
                table = driver.find_element(By.TAG_NAME, 'tbody') # 상품 테이블 찾기
                product_list = table.find_elements(By.XPATH, "//*[starts-with(@onclick, '$.nextPage(1,')]")  # 현재 페이지의 상품 개수 찾기
            else:
                print('ChagelistandCheck 출력 : 최초 페이지 리스트 개수 확인 ' + str(len(product_list)))
                break

        
        #time.sleep(2)
        print('ChagelistandCheck 출력 : 페이지 변경 후 상품개수 확인 시작')
        #print('if문시작')
        if page_now ==1: # 1페이지인경우 페이지 변경필요없이 진행
            pass
        else:            # 1패이지가 아닌경우 page_now에 저장되어있는 변수를 이용하여 해당 페이지로 접속
            print('ChagelistandCheck 출력 : ' + str(page_now) + '페이지로 변경')
            ##time.sleep(2)
            #pages = driver.find_element(By.CLASS_NAME, 'pg-wrap')
            #page_buttons = pages.find_elements(By.TAG_NAME, 'strong')+ pages.find_elements(By.TAG_NAME, 'a')
            #버튼 엘리멘트들이 모여있는 곳에서 순서대로 클릭하면 이전, 맨앞으로 버튼이있는 경우 숫자로 접속이 불가능하다
            #그러므로 버튼 엘리멘트들이 모여있는 곳의 엘리멘트를 전부 수집후 
            #해당 엘리멘트의 text가 page_now 변수가 일치하는지 순서대로 확인하여 일치하는 경우 클릭함
            while_count = 0
            #print('와일문시작')
            while while_count < 30:
                #print('트라이시작')
                try:
                    pages = driver.find_element(By.CSS_SELECTOR, '.pg-wrap, .paginate')
                    page_buttons = pages.find_elements(By.CSS_SELECTOR, 'strong, a')
                    
                    #page_buttons = pages.find_elements(By.TAG_NAME, 'strong')+ pages.find_elements(By.TAG_NAME, 'a')
                    #print(pages.text)
                    #print('포문시작')
                except:
                    print('ChagelistandCheck 출력 : 다음페이지로 넘어가는 버튼을 못찾음 1초후 재시도')
                    time.sleep(0.2)
                    # 페이지 번호 버튼이 갱신되었을 때 예외 처리
                    while_count += 1
                    continue  # 예외가 발생하면 다시 시도
                        #button.click()
                        ##time.sleep(1)
                        #break # 일치하여 클릭하면 다음 엘리멘트들은 확인할필요가 없기에 for문 중단

                page_true = 0
                for button in page_buttons: # 버튼 엘리멘트리스트에서 모든 엘리멘트에 대해서 for문 수행
                    if button.text == str(page_now): # 엘리멘트의 text와 현재 몇페이지인지 변수가 저장되어있는 page_now와 일치하는지 확인 후 클릭
                        button.click()
                        time.sleep(1)
                        #print(button.text)
                        #print('ChagelistandCheck 출력 : 다음페이지로 넘어가는 버튼 찾았음, 클릭 완료')
                        page_true = 1
                        break  # 클릭에 성공하면 루프를 종료
            
                    else:
                        #print(button.text)
                        #print('아님')
                        pass
                if page_true == 1:
                    print('ChagelistandCheck 출력 : ' + str(page_now) + '페이지있음 클릭 완료')
                    break
                else:
                    print('ChagelistandCheck 출력 : ' + str(page_now) + '페이지가 존재하지 않습니다. 확인 바랍니다.')
                    sys.exit()
                

        
        
        table = driver.find_element(By.TAG_NAME, 'tbody') # 상품 테이블 찾기
        product_list = table.find_elements(By.XPATH, "//*[starts-with(@onclick, '$.nextPage(1,')]")  # 현재 페이지의 상품 개수 찾기
        print('ChagelistandCheck 출력 : 현재 페이지 상품개수는 ' + str(len(product_list)) + '개 입니다.')
        #time.sleep(2)
        #print(range(len(product_list)))


        #print('왜안되는거야')
        return product_list
        #print('end')
        #print(product_list)
        #pass
            
        # except:
        #     print('ChagelistandCheck 출력 : 리스트개수변경못찾음 다시시도함???')
        #     if attempt < max_attempts - 4:
        #         continue  # 다시 시도
        #     else:
        #         print("ChagelistandCheck 출력 : 최대 시도회수 초과 코드종료!!!!!!")

In [ ]:
############## 알림창끄기 함수
def SaveHtml(fimename):
    html = driver.page_source #html 가져오기
    fimename = fimename + '.html'
    with open(fimename, "w", encoding="utf-8") as file:
        file.write(html)
SaveHtml('f12 안될때3')

In [ ]:
def start_chrome_dv():
    driver.get(fund_login) # 연금펀드 페이지 접속
    # 크롬 창을 창 모드로 최대화합니다
    driver.set_window_position(0, 0)
    driver.set_window_size(1800, 1024)  # 원하는 창 크기로 설정
    WB_IDPW()

### 예보이미지 부분

In [ ]:
#예보이미지
def find_protect_img(driver, classname, pd_name):
    # 엘리먼트 찾기
    try:
        element = driver.find_element(By.CLASS_NAME, classname)

        # 엘리먼트의 스타일 속성 값 가져오기
        style = element.value_of_css_property('background')
        

        # 스타일에서 이미지 URL 추출 (url(...) 형식)
        url_match = re.search(r'url\((.*?)\)', style)
        
    except:
        print('find_protect_img 출력 : 예보 이미지 경로가 없습니다.')
        image_url = ''
        file_name = ''
        return image_url, file_name
    if url_match:
        image_url = url_match.group(1).strip('"')
        
        # 이미지 URL을 절대 경로로 변환
        if image_url.startswith('/'):
            base_url = driver.current_url
            image_url = base_url + image_url

        # 이미지 다운로드
        response = requests.get(image_url)

        if response.status_code == 200:
            # 이미지를 저장
            file_name = os.path.join(protect_img, f'{pd_name}_예보문구.jpg')  # "예보문구"를 파일 이름에 추가
            with open(file_name, 'wb') as file:
                file.write(response.content)
            
            print('find_protect_img 출력 : 예보 이미지 저장 ************')
            return image_url, file_name

        else:
            print('find_protect_img 출력 : 예보 이미지를 가져오는데 문제가 있습니다.')
            image_url = ''
            file_name = ''
            return image_url, file_name
    else:
        print('find_protect_img 출력 : 예보 이미지 URL을 찾을 수 없습니다.')
        image_url = ''
        file_name = ''
        return image_url, file_name




### 상품이미지 부분

In [ ]:
# #상품이미지
# def find_product_imgs(driver, classname, pd_name):
#     elements = driver.find_elements(By.CLASS_NAME, classname)
#     # 각 엘리먼트에서 p 태그 안의 img src 속성 가져오기
#     for i in range(len(elements)):
#         p_tag = elements[i].find_element(By.TAG_NAME, 'p')
#         if p_tag:
#             try:
#                 img_tag = p_tag.find_element(By.TAG_NAME, 'img')
#                 #if img_tag and 'src' in img_tag.get_attribute('src'):
#                 img_url = img_tag.get_attribute('src')
#                 img_data = requests.get(img_url).content
#                 img_name = os.path.join(product_img, pd_name + str(i) + '번째_기본이미지명_'+ os.path.basename(img_url))
#                 with open(img_name, 'wb') as img_file:
#                     img_file.write(img_data)
#                 print("상품 이미지 저장이 완료되었습니다.")
#             except:
#                 image_url = ''
#                 img_name = ''
#                 print("상품 이미지가 없습니다.")
    
#     return image_url, img_name


In [ ]:
#상품이미지
def find_product_img(driver, classname, pd_name):
    try:
        elements = driver.find_element(By.CLASS_NAME, classname)
        # 각 엘리먼트에서 p 태그 안의 img src 속성 가져오기
        p_tag = elements.find_element(By.TAG_NAME, 'p')
    except:
        img_url = ''
        img_name = ''
        print('find_product_img 출력 : 상품 이미지 경로가 없습니다.')
        return img_url, img_name

    if p_tag:
        try:
            img_tag = p_tag.find_element(By.TAG_NAME, 'img')
            #if img_tag and 'src' in img_tag.get_attribute('src'):
            img_url = img_tag.get_attribute('src')
            img_data = requests.get(img_url).content
            img_name = os.path.join(product_img, pd_name + '_상품설명' + os.path.basename(img_url))
            with open(img_name, 'wb') as img_file:
                img_file.write(img_data)
            print('find_product_img 출력 : 상품 이미지 저장 ************')
            return img_url, img_name

        except:
            img_url = ''
            img_name = ''
            print('find_product_img 출력 : 상품 이미지가 없습니다.')
            return img_url, img_name

    
    

In [ ]:
# find_product_img(driver, 'owm-tab-con', 'ddd')

In [ ]:
# find_protect_img(driver, 'fnd_icon_nonGuaranteed', 'asdfff')

### 전체적인 크롤링 파트

In [ ]:

def wb_login_cr(url, filename, findbtnclick):
    findbtnclick = int(findbtnclick)
    inputfilename = filename
    crurl = url
    start_time_2 = time.time()  # 현재 시간 기록
    product_name = [] # 상품명 저장 상품들어가기 전에 초기화
    product_text = [] # 상품 요약 저장 상품들어가기전에 초기화
    product_texts = [] # 상품별 텍스트 수집 상품 들어가기 전에 초기화
    prodctu_tabs_check = [] # 탭수체크하기 상품 들어가기 전에 초기화
    product_noice = []
    
    CLCTS_ID = [] # 컬럼 공란
    INST_ID = [] # 기관아이디 KEB(하나), KB(국민), NH(농협), SH(신한), WB(우리) ID 1?
    GDS_INFO_CLCT_DT = [] # 상품정보수집일시 20230830-1058
    GDS_LCLSF_NM =[] # 상품대분류명 PERSONAL(개인), BUSINESS(기업), PUBLICNOTICE(상품공시)
    GDS_SCLSF_NM = [] # 상품소분류명 DEPOSIT MONEY(예금), FUND(펀드), TRUST(신탁), ISA(ISA), INSURANCE(보험), GOLD SILVER(골드/실버), RETIREMENT PENSION(퇴직연금)
    GDS_EXPLN_SE = [] # 상품설명구분 TEXT(상품안내페이지), PDF(상품설명서)
    GDS_NM = [] # 금융상품명
    GDS_EXPLN_CN = [] # 수집한 전체 텍스트
    DPSTR_PROT_GUDDOC = [] # 수집시 추출한 예금자보호안내문 없으면  NULL?
    IMG_STRG_PATH_NM = [] # 예금자보호 (이미지저장경로명), 없으면?
    IMG_NM = [] #@(이미지명),  없으면?
    GDS_EXPLN_URL = [] #(상품설명URL), 없으면?
    REG_DT = [] #(등록일시),   DEFAULT
    REG_USER = [] #(등록사용자),   DEFAULT
    MDFCN_DT = [] #(수정일시),   DEFAULT
    MDFCN_USER = [] #(수정사용자),   DEFAULT
    DEL_YN = [] #(삭제여부)  DEFAULT
    PDF_TEXT = []#pdf  
    PRODCUT_IMG_URL = []  #product_img_url, 
    PRODCUT_IMG_NAME = [] #product_img_name 
    driver.get(crurl) # 연금펀드 페이지 접속
    time.sleep(1)
    Alert()
    
    # 연금펀드 종류가 많음 한페이지당 20개리스트가 기본이나 100개로 변경가능하므로 100개씩보기
    
    
    
    listnum = 100  #
    page_now = 1  # 1페이지 변수 저장
    #product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)  # 최대 시도 횟수 지정 가능
    #time.sleep(1)
    ######################################################################
    ######################################################################
    ##########크롤링 실행##################################################
    ######################################################################
    ######################################################################
    #print("starttttt")
    #기본적으로 와일문으로 '다음페이지'와 관련된 엘리멘트가 없으면 마지막페이지로 인식 해당 페이지의 상품 크롤링이 끝나면 반복이 정지됨
    #test  product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)  # 최대 시도 횟수 지정 가능
    #test  time.sleep(1)
    total_pro_num = 1
    while True:  
        #listnum = 100  #
        #ChangeMaxList(listnum, max_attempts=5)
        #PageCheck(page_now)
        #####product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)
        #####print('페이지넘김')

        #table = driver.find_element(By.TAG_NAME, 'tbody') # 상품 테이블 찾기
        #product_list = table.find_elements(By.XPATH, "//*[starts-with(@onclick, '$.nextPage(1,')]")  # 현재 페이지의 상품 개수 찾기
        product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)  # 최대 시도 횟수 지정 가능
        #time.sleep(1)
        #print('이게맞아?')
        print('wb_login_cr 출력 : ' + str(range((len(product_list)))) + '범위의 상품 크롤링 시도')
        print('='*60)
        # 현재 페이지에 있는 상품 개수를 확인하여 해당 상품으로 들어가서 text 크롤링 시작하는 for문 시작
        for i in range(len(product_list)):
            print('wb_login_cr 출력 : ' + str(page_now) + '번째 페이지의 ' + str(i+1) + '번째 상품 크롤링 시작')
            print('='*60)
            start_time = time.time()  # 현재 시간 기록
                    
    
        
    
            # 상품 하나 크롤링 할때마다 페이지를 매번 계속 클릭해줘야하므로 위에 했던 페이지 클릭 코드 반복
            #listnum = 20  #
            #ChangeMaxList(listnum, max_attempts=5)
            #PageCheck(page_now)
            #test  product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)
            
            ######################################################################
            ######################################################################
            ##########상품명, 상품내용, 유의사항 저장################################
            ######################################################################
            ######################################################################
            #table = driver.find_element(By.TAG_NAME, 'tbody')   # 상품 테이블 찾기
            #product_list = table.find_elements(By.XPATH, "//*[starts-with(@onclick, '$.nextPage(1,')]")
            try:
                pd_name = product_list[i].text
                print('*'*60)
                print('wb_login_cr 출력 : 시작 : ' + pd_name)
                print('*'*60)
                product_name.append(pd_name)
                print('wb_login_cr 출력 : 상품명저장 ************')

                #### 예보이미지 저장
                protect_img_url, protect_img_name = find_protect_img(driver, 'fnd_icon_nonGuaranteed', pd_name)

                #time.sleep(1)
                
                
                try:
                    product_list[i].click()
                except :
                    print('wb_login_cr 출력 : 리스트못찾음 1초후에 다시 찾기')
                    #time.sleep(1)
                    product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)
                    table = driver.find_element(By.TAG_NAME, 'tbody')   # 상품 테이블 찾기
                    product_list = table.find_elements(By.XPATH, "//*[starts-with(@onclick, '$.nextPage(1,')]")
                    driver.execute_script("arguments[0].scrollIntoView();", product_list[i])
                    product_list[i].click()
        
                #time.sleep(1)
                product_class = driver.find_element(By.CLASS_NAME ,"owm-product")  # 상품내용있는 엘리멘트 선택
                list_text  = product_class.text  # 상품내용 저장
                if pd_name in list_text:  # 상품내용에서 반복저장된 상품명 제거
                    list_text_2 = list_text.replace(pd_name, "", 1)
                else:
                    list_text_2 = list_text
                    
                product_text.append(list_text_2)
                current_url = driver.current_url
                print('wb_login_cr 출력 : 상품요약저장 ************')

                ###상품이미지가 있을 경우 저장
                product_img_url, product_img_name = find_product_img(driver, 'owm-tab-con', pd_name)
                
                
                try: #유의사항이있는지 확인
                    notice = driver.find_element(By.ID ,"call_c027201")  # 유의사항이있는 엘리먼트 선택
                    list_notice = notice.text
                    product_noice.append(list_notice)
                    print('wb_login_cr 출력 : 유의사항저장 ************')
                except:
                    list_notice = '유의사항 없음'
                    product_noice.append('list_notice')
                    print('wb_login_cr 출력 : 유의사항없음저장 ************')
                #pdf url 저장하기
                product_pdf_name, pdf_list = gain_pdf_url(driver, 'info-down')
                #예금자 보호
                protect_list = driver.find_elements(By.CSS_SELECTOR,'#tabContent1 li')
                for protect in protect_list:
                    if '예금자' in protect.text:
                        protect_text = protect.text
                print('wb_login_cr 출력 : ' + protect_text)
                time.sleep(1)
                #상풍설명 (iframe -> url 새로 엶) - 없을 수도 있음.
                content = driver.find_element(By.CSS_SELECTOR, '#tabContent1').text#상품설명(예금자 보호문구있는쪽)
                ######################################################################
                ######################################################################
                ###########탭별정보가져오기############################################
                ######################################################################
                ######################################################################
        
                onclick_values = []   # 탭별 url들어있는 값 저장 상품들어갈때마다 초기화
                onclick_url = [] # 탭별 url만 뽑아서 저장 상품들어갈때마다 초기화
                #탭에있는 상품정보 가져오기
                #우리은행 펀드의 경우 탭이 여러개있고 그 탭의 정보는 text 크롤링이안되는경우가있음
                #각 탭별로 url이 있으며 그 url에 접속하여 내용을 크롤링하는경우와
                #url없이 바로 크롤링이 가능한 페이지가 있음(대부분 이미지로 설명되어있음 이부분은 이미지 다운 여부 확인필요)
        
                try:  # 먼저 url이 있는 경우
                    #time.sleep(1)
                    try:
                        element = driver.find_elements(By.ID, 'initContentView')
                        elements = driver.find_elements(By.CSS_SELECTOR , '[onclick*="clearIframe();insertIframe"]')
                    except:
                        elements = driver.find_elements(By.CSS_SELECTOR , '[onclick*="clearIframe();insertIframe"]')
                    elements = [ele for ele in (element + elements) if ele]
                    
                    for li_element in elements:
                        onclick_attribute = li_element.get_attribute('onclick')
                        if onclick_attribute:
                            onclick_values.append(onclick_attribute)
                    
                    
                    #위에서 수집한 탭별 url 정보가 있는 text에서 url만 추출
                    for value in onclick_values:
                        start_index = value.find("https://")
                        end_index = value.find("',", start_index)
                        url = value[start_index:end_index]
                        onclick_url.append(url)
                    
                    
                    driver.get(crurl)
                    time.sleep(1)
                    product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)  # 최대 시도 횟수 지정 가능
                    print('wb_login_cr 출력 : 페이지나오기완료')
                    #time.sleep(2)
                    Alert()    


                    page_texts = [] # 상품내 텍스트 수집 상품들어갈때마다 초기화
                    #counts_tabs = None # 상품내 탭수 수집 상품들어갈때마다 초기화
                    #위 두 리스트는 상품내 여러개의 탭에서 크롤링한 text를 저장했다가 합치는 용도로 한개의 상품크롤링이 끝나면 리스트내용을 하나의 스트링으로 합치고 상품내용, 탭수확인 리스트에 저장
                    
                    #위 에서 수집한 탭의 url에 접속하여 크롤링 시작
                    for j in range(len(onclick_url)):
                        #print('wb_login_cr 출력 : ' + str(j) + '번째 저장된 url의 내용 파싱 후 크롤링 중')
                        #driver.get(onclick_url[j])    
                        #time.sleep(0.5)  
                        try: # 경고창떴을때
                            alert = driver.switch_to.alert
                            alert.accept()
                        except:
                            pass
                        #url 접속 로딩 완료후 text 크롤링

                        # requests를 사용하여 웹 페이지 내용을 가져옴
                        response = requests.get(onclick_url[j])

                        # 응답의 내용을 Beautiful Soup으로 파싱
                        soup = BeautifulSoup(response.content, 'html.parser')

                        # 모든 텍스트를 가져와 출력
                        text_content = soup.get_text()
                        text_without_newlines = text_content.replace('\n', ' ')

                        # 결과 출력
                        #print(text_without_newlines)
                        #page_text = driver.find_element(By.TAG_NAME,'body').text
                        page_texts.append(text_without_newlines)
                        #print('wb_login_cr 출력 : ' + str(j) + '번째 크롤링 완료')
                        ##counts_tabs = j # 최대 탭수 저장 변수
                        #모든 탭 크롤링후 하나의 스트링으로 합치고 리스트에 append
                    #모든 탭 크롤링후 하나의 스트링으로 합치고 리스트에 append
                    print('wb_login_cr 출력 : 크롤링 완료')    

                except: # 상품설명이 이미지로되어있는경우(최신상품중에 있음)
                    print('이미지인 경우')
                    page_texts = [] # 상품내 텍스트 수집 상품들어갈때마다 초기화
                    # 이경우 탭이 여러개 있는 경우가 있지만 상품페이지에서 바로 text크롤링이 가능하기에 탭별로 url 수집 및 접속은 필요없음
                    elements = driver.find_elements(By.CSS_SELECTOR ,"[id^='tabContent']")
                    
                    for element in elements:
                        test = element.find_element(By.CLASS_NAME, 'owm-tab-con') # 탭별 엘리멘트 수집
                        if test.text == '': # 탭별 엘리멘트의 text가 없다면 탭이 활성화 안되어있는 경우로 get_attribute로 텍스트 크롤링
                            page_text = test.get_attribute("textContent")
                            page_texts.append(page_text)
                            print('wb_login_cr 출력 : 상품상세탭저장 ************')
                        else: # 탭별 엘리멘트에서 text가 바로 크롤링되는 경우
                            page_text = test.text 
                            page_texts.append(page_text)
                # 상품목록으로 다시 접속하여 다음상품 크롤링 준비
                    
                    driver.get(crurl)
                    time.sleep(1)
                    product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)  # 최대 시도 횟수 지정 가능
                    print('wb_login_cr 출력 : 페이지나오기완료')
                    #time.sleep(2)
                    Alert()       


                #combined_page_text = '\n'.join(page_texts) 
                #product_texts.append(combined_page_text)
                #print('wb_login_cr 출력 : 상품상세탭저장')
                #prodctu_tabs_check.append('총 탭수 : ' + str(counts_tabs))
                #print('wb_login_cr 출력 : 탭수저장')
                #print('wb_login_cr 출력 : 완료 : ' + pd_name)                
                
                
                #모든 탭 크롤링후 하나의 스트링으로 합치고 리스트에 append
                combined_page_text = '\n'.join(page_texts) 
                product_texts.append(combined_page_text)
                print('wb_login_cr 출력 : 상품상세탭저장 ************')
                #prodctu_tabs_check.append('총 탭수 : ' + str(len(elements)))
                # print('wb_login_cr 출력 : 탭수저장')
                pdf_text, pdf_protect_text = cr_pdf(product_pdf_name, pdf_list)
                print('wb_login_cr 출력 : PDF 상품내용 저장 ************')
                print('wb_login_cr 출력 : 완료 : ' + pd_name)
        
        
                now = datetime.now()
                # 원하는 형식으로 시간을 포맷합니다.
                formatted_time = now.strftime("%Y%m%d-%H%M")
                CLCTS_ID.append('')
                INST_ID.append('WB')  # 기관아이디 KEB(하나), KB(국민), NH(농협), SH(신한), WB(우리) ID 1?
                GDS_INFO_CLCT_DT.append(formatted_time) # 상품정보수집일시 20230830-1058
                GDS_LCLSF_NM.append('PERSONAL') # 상품대분류명 PERSONAL(개인), BUSINESS(기업), PUBLICNOTICE(상품공시)
                GDS_SCLSF_NM.append('FUND') # 상품소분류명 DEPOSIT MONEY(예금), FUND(펀드), TRUST(신탁), ISA(ISA), INSURANCE(보험), GOLD SILVER(골드/실버), RETIREMENT PENSION(퇴직연금)
                GDS_EXPLN_SE.append('TEXT') # 상품설명구분 TEXT(상품안내페이지), PDF(상품설명서)
                GDS_NM.append(pd_name) # 금융상품명
                gds_text = content + list_text_2 + list_notice + combined_page_text
                GDS_EXPLN_CN.append(gds_text) # 수집한 전체 텍스트
                DPSTR_PROT_GUDDOC.append(protect_text)  # 수집시 추출한 예금자보호안내문 없으면  NULL
                IMG_STRG_PATH_NM.append(protect_img_url)  # 예금자보호 (이미지저장경로명), 없으면?protect_img_url, protect_img_name
                IMG_NM.append(protect_img_name)#@(이미지명),  없으면?
                GDS_EXPLN_URL.append(current_url) #(상품설명URL), 없으면?
                REG_DT.append('DEFAULT') #(등록일시),   DEFAULT
                REG_USER.append('DEFAULT')  #(등록사용자),   DEFAULT
                MDFCN_DT.append('DEFAULT')  #(수정일시),   DEFAULT
                MDFCN_USER.append('DEFAULT') #(수정사용자),   DEFAULT
                DEL_YN.append('DEFAULT')  #(삭제여부)  DEFAULT
                #PDF_TEXT.append(pdf_text)  #pdf  DEFAULT product_img_url, product_img_name 
                PRODCUT_IMG_URL.append(product_img_url)  #pdf  DEFAULT product_img_url, product_img_name 
                PRODCUT_IMG_NAME.append(product_img_name)  #pdf  DEFAULT product_img_url, product_img_name 





                CLCTS_ID.append('')
                INST_ID.append('WB')  # 기관아이디 KEB(하나), KB(국민), NH(농협), SH(신한), WB(우리) ID 1?
                GDS_INFO_CLCT_DT.append(formatted_time) # 상품정보수집일시 20230830-1058
                GDS_LCLSF_NM.append('PERSONAL') # 상품대분류명 PERSONAL(개인), BUSINESS(기업), PUBLICNOTICE(상품공시)
                GDS_SCLSF_NM.append('FUND') # 상품소분류명 DEPOSIT MONEY(예금), FUND(펀드), TRUST(신탁), ISA(ISA), INSURANCE(보험), GOLD SILVER(골드/실버), RETIREMENT PENSION(퇴직연금)
                GDS_EXPLN_SE.append('PDF') # 상품설명구분 TEXT(상품안내페이지), PDF(상품설명서)
                GDS_NM.append(pd_name) # 금융상품명
                GDS_EXPLN_CN.append(pdf_text) # 수집한 전체 텍스트
                DPSTR_PROT_GUDDOC.append(pdf_protect_text)  # 수집시 추출한 예금자보호안내문 없으면  NULL
                IMG_STRG_PATH_NM.append(protect_img_url)  # 예금자보호 (이미지저장경로명), 없으면?protect_img_url, protect_img_name
                IMG_NM.append(protect_img_name)#@(이미지명),  없으면?
                GDS_EXPLN_URL.append(current_url) #(상품설명URL), 없으면?
                REG_DT.append('DEFAULT') #(등록일시),   DEFAULT
                REG_USER.append('DEFAULT')  #(등록사용자),   DEFAULT
                MDFCN_DT.append('DEFAULT')  #(수정일시),   DEFAULT
                MDFCN_USER.append('DEFAULT') #(수정사용자),   DEFAULT
                DEL_YN.append('DEFAULT')  #(삭제여부)  DEFAULT
                #PDF_TEXT.append(pdf_text)  #pdf  DEFAULT product_img_url, product_img_name 
                PRODCUT_IMG_URL.append(product_img_url)  #pdf  DEFAULT product_img_url, product_img_name 
                PRODCUT_IMG_NAME.append(product_img_name)  #pdf  DEFAULT product_img_url, product_img_name 







        
                
                
                # 상품목록으로 다시 접속하여 다음상품 크롤링 준비
                #driver.get(crurl) 
                #print('wb_login_cr 출력 : 페이지나오기완료')
                #time.sleep(2)
                #Alert()
                #time.sleep(1)
                
                #상품목록으로 돌아오면 100리스트로 저장해두었던 리스트가 20리스트로 다시 줄어들어있으므로 다시 100리스트로 변경
                #time.sleep(1)
                #listnum = 100  #
                #product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)  # 최대 시도 횟수 지정 가능   
                Alert()
                end_time = time.time()    # 작업 종료 후 시간 기록
                elapsed_time = end_time - start_time
                sum_time = end_time - start_time_2
                mean_time = sum_time / total_pro_num
                print('wb_login_cr 출력 : ' + f"{total_pro_num} 번째 상품 크롤링 소요시간 : {elapsed_time} 초, 누적 소요시간 : {sum_time} 초, 평균 소요시간 : {mean_time} 초")
                print('='*60)
                # 1개 상품 크롤링 종료
                total_pro_num += 1
            except Exception as e:
                # 모든 종류의 예외에 대한 처리 코드
                print("에러 발생:", e)
        # 페이지에있는 모등 상품 크롤링 종료
        # 역시 이때도 상품목록으로 돌아온 상태이므로 페이지가 초기화되므로 페이지에 다시 접속해야함
        print('wb_login_cr 출력 : ' + str(page_now) + '페이지 크롤링 완료. 다음 페이지 확인')
        time.sleep(1)
        product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)
                
        # 페이지 정보탭에 '다음'이라는 text 확인후 있으면 다음페이지가 있는 것이므로 page_now를 1증가, 없다면 마지막페이지인것이므로 와일문 break
        # '다음'이라는 text를 찾는 로직은 페이지를 클릭하는 코드부분과 동일함
        # 페이지버튼박스 클래스네임인 'paginate'과 다음버튼의 text인 '다음'을 변경하면 와일문과 조합하여 무한반복으로 사용가능할듯
        #PageCheck(page_now)
        #ChagelistandCheck(listnum, 5, page_now, findbtnclick)
        time.sleep(2)
        pages = driver.find_element(By.CSS_SELECTOR, '.pg-wrap, .paginate')
        next_button = pages.find_element(By.XPATH, "//*[contains(text(), '다음')]")
        next_button.text
        if next_button.text == '':
            print('wb_login_cr 출력 : 마지막페이지')
            print(' ')
            break
        else:
            page_now += 1
            print('wb_login_cr 출력 : 페이지증가')
            print(' ')
            
    data = {
        #'상품명': product_name,
        #'상품요약': product_text,
        #'상품내용': product_texts,
        #'유의사항': product_noice,
        #'탭수확인': prodctu_tabs_check,
        'CLCTS_ID' : CLCTS_ID,
        'INST_ID' : INST_ID,
        'GDS_INFO_CLCT_DT' : GDS_INFO_CLCT_DT,
        'GDS_LCLSF_NM' : GDS_LCLSF_NM,
        'GDS_SCLSF_NM' : GDS_SCLSF_NM,
        'GDS_EXPLN_SE' : GDS_EXPLN_SE,
        'GDS_NM' : GDS_NM,
        'GDS_EXPLN_CN' : GDS_EXPLN_CN,
        'DPSTR_PROT_GUDDOC' : DPSTR_PROT_GUDDOC,
        'IMG_STRG_PATH_NM' : IMG_STRG_PATH_NM,
        'IMG_NM' : IMG_NM,
        'GDS_EXPLN_URL' : GDS_EXPLN_URL,
        'REG_DT' : REG_DT,
        'REG_USER' : REG_USER,
        'MDFCN_DT' : MDFCN_DT,
        'MDFCN_USER' : MDFCN_USER,
        'DEL_YN' : DEL_YN,
        #'PDF_TEXT' : PDF_TEXT,
        'PRODCUT_IMG_URL' : PRODCUT_IMG_URL,
        'PRODCUT_IMG_NAME' : PRODCUT_IMG_NAME,
    }
      
    
    
    
    end_time_2 = time.time()    # 작업 종료 후 시간 기록
    elapsed_time_2 = end_time_2 - start_time_2
    mean_time_2 = elapsed_time_2 / (total_pro_num - 1)
    print('wb_login_cr 출력 : ' + f"총 소요시간 : {elapsed_time_2} 초, 평균 소요시간 : {mean_time_2} 초")
    # 딕셔너리를 데이터프레임으로 변환
    df = pd.DataFrame(data)
    # 데이터프레임 출력
    df.to_csv(inputfilename, sep=',', encoding = 'utf-8-sig',index=None)
    return df


In [ ]:
#우리은행로그인 프로그램 실행 아이디 비번 입력후 로그인후 크롤링 버튼누르면 크롤링 시작
start_chrome_dv()

표준 출력:


표준 에러 출력:

종료 코드: 0


In [ ]:
df_fund_1 = wb_login_cr(fund_url_1, '상세펀드.csv', 0)
df_fund_1.to_csv('상세펀드2.csv', sep=',', encoding = 'utf-8-sig',index=None)

ChagelistandCheck 출력 : 리스트100개로 변경 완료
ChagelistandCheck 출력 : 최초 페이지 리스트 개수 확인 100
ChagelistandCheck 출력 : 페이지 변경 후 상품개수 확인 시작
ChagelistandCheck 출력 : 현재 페이지 상품개수는 100개 입니다.
wb_login_cr 출력 : range(0, 100)범위의 상품 크롤링 시도
wb_login_cr 출력 : 1번째 페이지의 1번째 상품 크롤링 시작
************************************************************
wb_login_cr 출력 : 시작 : 다올VIP스타셀렉션증권자투자신탁(주식)Ae
************************************************************
wb_login_cr 출력 : 상품명저장 ************
find_protect_img 출력 : 예보 이미지 저장 ************
wb_login_cr 출력 : 상품요약저장 ************
find_product_img 출력 : 상품 이미지가 없습니다.
wb_login_cr 출력 : 유의사항저장 ************
wb_login_cr 출력 : 이 금융상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.
ChagelistandCheck 출력 : 리스트100개로 변경 완료
ChagelistandCheck 출력 : 최초 페이지 리스트 개수 확인 100
ChagelistandCheck 출력 : 페이지 변경 후 상품개수 확인 시작
ChagelistandCheck 출력 : 현재 페이지 상품개수는 100개 입니다.
wb_login_cr 출력 : 페이지나오기완료
wb_login_cr 출력 : 크롤링 완료
wb_login_cr 출력 : 상품상세탭저장 ************
cr_pdf 출력 : PDF 있음 크롤링 함수 시작
 또한 이 집합투자증권은「예금자보호법
cr_pdf 출력 : PDF 크롤링 완료

In [ ]:
df_fund_2 = wb_login_cr(fund_url_2, '연금펀드.csv', 1)
df_fund_2.to_csv('연금펀드2.csv', sep=',', encoding = 'utf-8-sig',index=None)

In [ ]:
display(df_fund_1)

,CLCTS_ID,INST_ID,GDS_INFO_CLCT_DT,GDS_LCLSF_NM,GDS_SCLSF_NM,GDS_EXPLN_SE,GDS_NM,GDS_EXPLN_CN,DPSTR_PROT_GUDDOC,IMG_STRG_PATH_NM,IMG_NM,GDS_EXPLN_URL,REG_DT,REG_USER,MDFCN_DT,MDFCN_USER,DEL_YN,PRODCUT_IMG_URL,PRODCUT_IMG_NAME
0,,WB,20230902-1439,PERSONAL,FUND,TEXT,미래에셋달러우량중장기채권증권자투자신탁(UH)(채권)C-e,개요\n이 투자신탁은 미래에셋달러우량중장기채권증권모투자신탁(채권)에 80% 이상 투...,이 금융상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\미래에셋달러우량중장기채권증권자투자신탁(UH)(채권)C...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
1,,WB,20230902-1439,PERSONAL,FUND,PDF,미래에셋달러우량중장기채권증권자투자신탁(UH)(채권)C-e,집합투자규약미래에셋달러우량중장기채권증권자투자신탁(UH)(채권)신탁계약서집합투자업자 ...,또한 이 집합투자증권은 「예금자보호법」에 의한 보호를 받지 않는 실적배당상품으로 투...,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\미래에셋달러우량중장기채권증권자투자신탁(UH)(채권)C...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
2,,WB,20230902-1439,PERSONAL,FUND,TEXT,우리GPIMCO분산투자증권자투자신탁[채권_재간접형](H)A-e,"개요\n이 투자신탁은 외국집합투자기구인 "" PIMCO Funds GIS plc ; ...",이 금융상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\우리GPIMCO분산투자증권자투자신탁[채권_재간접형](...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
3,,WB,20230902-1439,PERSONAL,FUND,PDF,우리GPIMCO분산투자증권자투자신탁[채권_재간접형](H)A-e,집합투자규약 1우우리리GG PPIIMMCCOO분분산산투투자자증증권권자자투투자자신...,집 합투자증권은 ｢예금자보호법 ｣에 따라 예금보험공사가 보호하지않는 실적배당상...,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\우리GPIMCO분산투자증권자투자신탁[채권_재간접형](...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
4,,WB,20230902-1439,PERSONAL,FUND,TEXT,하나UBSPIMCO글로벌투자등급증권자투자신탁(채권-재간접형)A-E,개요\n이 투자신탁은 해외 채권에 투자하는 집합투자증권에 주로 투자하는 모투자신탁에...,이 금융상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\하나UBSPIMCO글로벌투자등급증권자투자신탁(채권-재...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
5,,WB,20230902-1439,PERSONAL,FUND,PDF,하나UBSPIMCO글로벌투자등급증권자투자신탁(채권-재간접형)A-E,집합투자규약 하나UBS PIMCO 글로벌투자등급증권자투자신탁 [채권-재간접형 ] ...,또한 이 집합투자증권은 「예금자보호법」에 따라 예금보험공사가 보호하지 않는...,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\하나UBSPIMCO글로벌투자등급증권자투자신탁(채권-재...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
6,,WB,20230902-1440,PERSONAL,FUND,TEXT,하나UBSPIMCO글로벌투자등급증권자투자신탁(채권-재간접형)C-E,개요\n이 투자신탁은 해외 채권에 투자하는 집합투자증권에 주로 투자하는 모투자신탁에...,이 금융상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\하나UBSPIMCO글로벌투자등급증권자투자신탁(채권-재...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
7,,WB,20230902-1440,PERSONAL,FUND,PDF,하나UBSPIMCO글로벌투자등급증권자투자신탁(채권-재간접형)C-E,집합투자규약 하나UBS PIMCO 글로벌투자등급증권자투자신탁 [채권-재간접형 ] ...,또한 이 집합투자증권은 「예금자보호법」에 따라 예금보험공사가 보호하지 않는...,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\하나UBSPIMCO글로벌투자등급증권자투자신탁(채권-재...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
8,,WB,20230902-1440,PERSONAL,FUND,TEXT,우리GPIMCO글로벌투자등급증권자투자신탁[채권_재간접형](H)A-e,"개요\n전세계 투자등급 회사채에 주로 투자하여 국가, 섹터 등에 분산투자 효과 추구...",이 금융상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\우리GPIMCO글로벌투자등급증권자투자신탁[채권_재간접...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,
9,,WB,20230902-1440,PERSONAL,FUND,PDF,우리GPIMCO글로벌투자등급증권자투자신탁[채권_재간접형](H)A-e,집합투자규약 1 우우리리GG PPIIMMCCOO글글로로벌벌투투자자등등급급증증권권...,집합투자증권은 ｢예금자보호법 ｣에 따라 예금보험공사가 보호하지 않는 실적배당...,https://spot.wooribank.com/img/owm/ico_non_gua...,protect_fund_img\우리GPIMCO글로벌투자등급증권자투자신탁[채권_재간접...,https://spot.wooribank.com/pot/Dream?withyou=O...,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,,


In [ ]:
# df_fund_2 = pd.read_csv('연금펀드2.csv')

In [ ]:
# df_fund_2['GDS_EXPLN_CN']

In [ ]:
# # 중복된 문장 찾기
# duplicate_sentences = df_fund_2['GDS_EXPLN_CN'][1].split('.')

# # 중복된 문장 출력
# for sentence in duplicate_sentences:
#     if df_fund_2['GDS_EXPLN_CN'][1].count(sentence) > 1:
#         print("중복된 문장:", sentence)

In [ ]:
# # 예금자 보호 또는 예금자보호를 포함하는 행 찾기
# contains_deposit_protection = df_fund_2['GDS_EXPLN_CN'].str.contains(r'예금자\s*보호|예금자보호')

# # 결과 출력
# print(contains_deposit_protection)

In [ ]:
# import pandas as pd

# # 예금자 보호 또는 예금자보호를 포함하는 행 찾기
# contains_deposit_protection = df_fund_2[df_fund_2['GDS_EXPLN_CN'].str.contains(r'예금자\s*보호|예금자보호')]

# # 결과 출력
# for index, row in contains_deposit_protection.iterrows():
#     print(row['GDS_EXPLN_CN'])

In [ ]:
# # 'PDF_TEXT' 열에서 '예금자 보호' 또는 '예금자보호'를 포함하는 문장을 추출하고 출력
# for text in df_fund_2['GDS_EXPLN_CN']:
#     print(type(text))

In [ ]:
# # 'PDF_TEXT' 열에서 '예금자 보호' 또는 '예금자보호'를 포함하는 문장을 추출하고 출력
# for text in df_fund_2['GDS_EXPLN_CN']:
#     sentences = re.split(r'(?<=[.!?])', text)
#     for sentence in sentences:
#         if re.search(r'(?:예금자\s*보호|예금자보호)', sentence):
#             print(sentence)
#             print('')

In [ ]:
# df_fund_2['PDF_TEXT'].iloc[0]

In [ ]:
# for text in df_fund_2['PDF_TEXT']:
#     sentences = re.split(r'(?<=[.!?])', text)
#     print(sentences)

In [ ]:
# findbtnclick = 1
# inputfilename = 'tes22t.csv'
# # crurl = url
# start_time_2 = time.time()  # 현재 시간 기록
# product_name = [] # 상품명 저장 상품들어가기 전에 초기화
# product_text = [] # 상품 요약 저장 상품들어가기전에 초기화
# product_texts = [] # 상품별 텍스트 수집 상품 들어가기 전에 초기화
# prodctu_tabs_check = [] # 탭수체크하기 상품 들어가기 전에 초기화
# product_noice = []

# CLCTS_ID = [] # 컬럼 공란
# INST_ID = [] # 기관아이디 KEB(하나), KB(국민), NH(농협), SH(신한), WB(우리) ID 1?
# GDS_INFO_CLCT_DT = [] # 상품정보수집일시 20230830-1058
# GDS_LCLSF_NM =[] # 상품대분류명 PERSONAL(개인), BUSINESS(기업), PUBLICNOTICE(상품공시)
# GDS_SCLSF_NM = [] # 상품소분류명 DEPOSIT MONEY(예금), FUND(펀드), TRUST(신탁), ISA(ISA), INSURANCE(보험), GOLD SILVER(골드/실버), RETIREMENT PENSION(퇴직연금)
# GDS_EXPLN_SE = [] # 상품설명구분 TEXT(상품안내페이지), PDF(상품설명서)
# GDS_NM = [] # 금융상품명
# GDS_EXPLN_CN = [] # 수집한 전체 텍스트
# DPSTR_PROT_GUDDOC = [] # 수집시 추출한 예금자보호안내문 없으면  NULL?
# IMG_STRG_PATH_NM = [] # 예금자보호 (이미지저장경로명), 없으면?
# IMG_NM = [] #@(이미지명),  없으면?
# GDS_EXPLN_URL = [] #(상품설명URL), 없으면?
# REG_DT = [] #(등록일시),   DEFAULT
# REG_USER = [] #(등록사용자),   DEFAULT
# MDFCN_DT = [] #(수정일시),   DEFAULT
# MDFCN_USER = [] #(수정사용자),   DEFAULT
# DEL_YN = [] #(삭제여부)  DEFAULT
# PDF_TEXT = []#pdf  
# PRODCUT_IMG_URL = []  #product_img_url, 
# PRODCUT_IMG_NAME = [] #product_img_name 




# pd_name = 'aaa'
# product_class = driver.find_element(By.CLASS_NAME ,"owm-product")  # 상품내용있는 엘리멘트 선택
# list_text  = product_class.text  # 상품내용 저장
# if pd_name in list_text:  # 상품내용에서 반복저장된 상품명 제거
#     list_text_2 = list_text.replace(pd_name, "", 1)
# else:
#     list_text_2 = list_text
    
# product_text.append(list_text_2)
# current_url = driver.current_url
# print('wb_login_cr 출력 : 상품요약저장')

# ###상품이미지가 있을 경우 저장
# product_img_url, product_img_name = find_product_img(driver, 'owm-tab-con', pd_name)


# try: #유의사항이있는지 확인
#     notice = driver.find_element(By.ID ,"call_c027201")  # 유의사항이있는 엘리먼트 선택
#     list_notice = notice.text
#     product_noice.append(list_notice)
#     print('wb_login_cr 출력 : 유의사항저장')
# except:
#     list_notice = '유의사항 없음'
#     product_noice.append('list_notice')
#     print('wb_login_cr 출력 : 유의사항없음저장')
# #pdf url 저장하기
# product_pdf_name, pdf_list = gain_pdf_url(driver, 'info-down')
# #예금자 보호
# protect_list = driver.find_elements(By.CSS_SELECTOR,'#tabContent1 li')
# for protect in protect_list:
#     if '예금자' in protect.text:
#         protect_text = protect.text
# print(protect_text)
# time.sleep(1)
# #상풍설명 (iframe -> url 새로 엶) - 없을 수도 있음.
# content = driver.find_element(By.CSS_SELECTOR, '#tabContent1').text#상품설명(예금자 보호문구있는쪽)
# ######################################################################
# ######################################################################
# ###########탭별정보가져오기############################################
# ######################################################################
# ######################################################################

# onclick_values = []   # 탭별 url들어있는 값 저장 상품들어갈때마다 초기화
# onclick_url = [] # 탭별 url만 뽑아서 저장 상품들어갈때마다 초기화
# #탭에있는 상품정보 가져오기
# #우리은행 펀드의 경우 탭이 여러개있고 그 탭의 정보는 text 크롤링이안되는경우가있음
# #각 탭별로 url이 있으며 그 url에 접속하여 내용을 크롤링하는경우와
# #url없이 바로 크롤링이 가능한 페이지가 있음(대부분 이미지로 설명되어있음 이부분은 이미지 다운 여부 확인필요)

# try:  # 먼저 url이 있는 경우
#     #time.sleep(1)
#     try:
#         element = driver.find_elements(By.ID, 'initContentView')
#         elements = driver.find_elements(By.CSS_SELECTOR , '[onclick*="clearIframe();insertIframe"]')
#     except:
#         elements = driver.find_elements(By.CSS_SELECTOR , '[onclick*="clearIframe();insertIframe"]')
#     elements = [ele for ele in (element + elements) if ele]
    
#     for li_element in elements:
#         onclick_attribute = li_element.get_attribute('onclick')
#         if onclick_attribute:
#             onclick_values.append(onclick_attribute)
    
    
#     #위에서 수집한 탭별 url 정보가 있는 text에서 url만 추출
#     for value in onclick_values:
#         start_index = value.find("https://")
#         end_index = value.find("',", start_index)
#         url = value[start_index:end_index]
#         onclick_url.append(url)
    
    
#     #driver.get(crurl)
#     time.sleep(1)
#     #product_list = ChagelistandCheck(listnum, 5, page_now, findbtnclick)  # 최대 시도 횟수 지정 가능
#     print('wb_login_cr 출력 : 페이지나오기완료')
#     #time.sleep(2)
#     Alert()    


#     page_texts = [] # 상품내 텍스트 수집 상품들어갈때마다 초기화
#     #counts_tabs = None # 상품내 탭수 수집 상품들어갈때마다 초기화
#     #위 두 리스트는 상품내 여러개의 탭에서 크롤링한 text를 저장했다가 합치는 용도로 한개의 상품크롤링이 끝나면 리스트내용을 하나의 스트링으로 합치고 상품내용, 탭수확인 리스트에 저장
    
#     #위 에서 수집한 탭의 url에 접속하여 크롤링 시작
#     for j in range(len(onclick_url)):
#         print('wb_login_cr 출력 : ' + str(j) + '번째 저장된 url의 내용 파싱 후 크롤링 중')
#         #driver.get(onclick_url[j])    
#         #time.sleep(0.5)  
#         try: # 경고창떴을때
#             alert = driver.switch_to.alert
#             alert.accept()
#         except:
#             pass
#         #url 접속 로딩 완료후 text 크롤링

#         # requests를 사용하여 웹 페이지 내용을 가져옴
#         response = requests.get(onclick_url[j])

#         # 응답의 내용을 Beautiful Soup으로 파싱
#         soup = BeautifulSoup(response.content, 'html.parser')

#         # 모든 텍스트를 가져와 출력
#         text_content = soup.get_text()
#         text_without_newlines = text_content.replace('\n', ' ')

#         # 결과 출력
#         #print(text_without_newlines)
#         #page_text = driver.find_element(By.TAG_NAME,'body').text
#         page_texts.append(text_without_newlines)
#         print('wb_login_cr 출력 : ' + str(j) + '번째 크롤링 완료')
#         #counts_tabs = j # 최대 탭수 저장 변수
#         #모든 탭 크롤링후 하나의 스트링으로 합치고 리스트에 append
#     #모든 탭 크롤링후 하나의 스트링으로 합치고 리스트에 append
        

# except: # 상품설명이 이미지로되어있는경우(최신상품중에 있음)
#     print('이미지인 경우')
#     page_texts = [] # 상품내 텍스트 수집 상품들어갈때마다 초기화
#     # 이경우 탭이 여러개 있는 경우가 있지만 상품페이지에서 바로 text크롤링이 가능하기에 탭별로 url 수집 및 접속은 필요없음
#     elements = driver.find_elements(By.CSS_SELECTOR ,"[id^='tabContent']")
    
#     for element in elements:
#         test = element.find_element(By.CLASS_NAME, 'owm-tab-con') # 탭별 엘리멘트 수집
#         if test.text == '': # 탭별 엘리멘트의 text가 없다면 탭이 활성화 안되어있는 경우로 get_attribute로 텍스트 크롤링
#             page_text = test.get_attribute("textContent")
#             page_texts.append(page_text)
#             print('wb_login_cr 출력 : 상품상세탭저장')
#         else: # 탭별 엘리멘트에서 text가 바로 크롤링되는 경우
#             page_text = test.text 
#             page_texts.append(page_text)
# # 상품목록으로 다시 접속하여 다음상품 크롤링 준비
    
#     driver.get(crurl) 
#     print('wb_login_cr 출력 : 페이지나오기완료')
#     #time.sleep(2)
#     Alert()    


# #combined_page_text = '\n'.join(page_texts) 
# #product_texts.append(combined_page_text)
# #print('wb_login_cr 출력 : 상품상세탭저장')
# #prodctu_tabs_check.append('총 탭수 : ' + str(counts_tabs))
# #print('wb_login_cr 출력 : 탭수저장')
# #print('wb_login_cr 출력 : 완료 : ' + pd_name)                


# #모든 탭 크롤링후 하나의 스트링으로 합치고 리스트에 append
# combined_page_text = '\n'.join(page_texts) 
# product_texts.append(combined_page_text)
# print('wb_login_cr 출력 : 상품상세탭저장')
# #prodctu_tabs_check.append('총 탭수 : ' + str(len(elements)))
# # print('wb_login_cr 출력 : 탭수저장')
# pdf_text, pdf_protect_text = cr_pdf(product_pdf_name, pdf_list)
# print('wb_login_cr 출력 : PDF전부 저장완료')
# print('wb_login_cr 출력 : 완료 : ' + pd_name)


# now = datetime.now()
# # 원하는 형식으로 시간을 포맷합니다.
# formatted_time = now.strftime("%Y%m%d-%H%M")
# CLCTS_ID.append('')
# INST_ID.append('WB')  # 기관아이디 KEB(하나), KB(국민), NH(농협), SH(신한), WB(우리) ID 1?
# GDS_INFO_CLCT_DT.append(formatted_time) # 상품정보수집일시 20230830-1058
# GDS_LCLSF_NM.append('PERSONAL') # 상품대분류명 PERSONAL(개인), BUSINESS(기업), PUBLICNOTICE(상품공시)
# GDS_SCLSF_NM.append('FUND') # 상품소분류명 DEPOSIT MONEY(예금), FUND(펀드), TRUST(신탁), ISA(ISA), INSURANCE(보험), GOLD SILVER(골드/실버), RETIREMENT PENSION(퇴직연금)
# GDS_EXPLN_SE.append('TEXT') # 상품설명구분 TEXT(상품안내페이지), PDF(상품설명서)
# GDS_NM.append(pd_name) # 금융상품명
# gds_text = content + list_text_2 + list_notice + combined_page_text
# GDS_EXPLN_CN.append(gds_text) # 수집한 전체 텍스트
# DPSTR_PROT_GUDDOC.append(protect_text)  # 수집시 추출한 예금자보호안내문 없으면  NULL
# IMG_STRG_PATH_NM.append(protect_img_url)  # 예금자보호 (이미지저장경로명), 없으면?protect_img_url, protect_img_name
# IMG_NM.append(protect_img_name)#@(이미지명),  없으면?
# GDS_EXPLN_URL.append(current_url) #(상품설명URL), 없으면?
# REG_DT.append('DEFAULT') #(등록일시),   DEFAULT
# REG_USER.append('DEFAULT')  #(등록사용자),   DEFAULT
# MDFCN_DT.append('DEFAULT')  #(수정일시),   DEFAULT
# MDFCN_USER.append('DEFAULT') #(수정사용자),   DEFAULT
# DEL_YN.append('DEFAULT')  #(삭제여부)  DEFAULT
# #PDF_TEXT.append(pdf_text)  #pdf  DEFAULT product_img_url, product_img_name 
# PRODCUT_IMG_URL.append(product_img_url)  #pdf  DEFAULT product_img_url, product_img_name 
# PRODCUT_IMG_NAME.append(product_img_name)  #pdf  DEFAULT product_img_url, product_img_name 





# CLCTS_ID.append('')
# INST_ID.append('WB')  # 기관아이디 KEB(하나), KB(국민), NH(농협), SH(신한), WB(우리) ID 1?
# GDS_INFO_CLCT_DT.append(formatted_time) # 상품정보수집일시 20230830-1058
# GDS_LCLSF_NM.append('PERSONAL') # 상품대분류명 PERSONAL(개인), BUSINESS(기업), PUBLICNOTICE(상품공시)
# GDS_SCLSF_NM.append('FUND') # 상품소분류명 DEPOSIT MONEY(예금), FUND(펀드), TRUST(신탁), ISA(ISA), INSURANCE(보험), GOLD SILVER(골드/실버), RETIREMENT PENSION(퇴직연금)
# GDS_EXPLN_SE.append('PDF') # 상품설명구분 TEXT(상품안내페이지), PDF(상품설명서)
# GDS_NM.append(pd_name) # 금융상품명
# GDS_EXPLN_CN.append(pdf_text) # 수집한 전체 텍스트
# DPSTR_PROT_GUDDOC.append(pdf_protect_text)  # 수집시 추출한 예금자보호안내문 없으면  NULL
# IMG_STRG_PATH_NM.append(protect_img_url)  # 예금자보호 (이미지저장경로명), 없으면?protect_img_url, protect_img_name
# IMG_NM.append(protect_img_name)#@(이미지명),  없으면?
# GDS_EXPLN_URL.append(current_url) #(상품설명URL), 없으면?
# REG_DT.append('DEFAULT') #(등록일시),   DEFAULT
# REG_USER.append('DEFAULT')  #(등록사용자),   DEFAULT
# MDFCN_DT.append('DEFAULT')  #(수정일시),   DEFAULT
# MDFCN_USER.append('DEFAULT') #(수정사용자),   DEFAULT
# DEL_YN.append('DEFAULT')  #(삭제여부)  DEFAULT
# #PDF_TEXT.append(pdf_text)  #pdf  DEFAULT product_img_url, product_img_name 
# PRODCUT_IMG_URL.append(product_img_url)  #pdf  DEFAULT product_img_url, product_img_name 
# PRODCUT_IMG_NAME.append(product_img_name)  #pdf  DEFAULT product_img_url, product_img_name 






wb_login_cr 출력 : 상품요약저장
find_product_img 출력 : 이미지가 없습니다.
wb_login_cr 출력 : 유의사항저장
이 금융상품은 예금자보호법에 따라 예금보험공사가 보호하지 않습니다.
wb_login_cr 출력 : 페이지나오기완료
wb_login_cr 출력 : 상품상세탭저장
cr_pdf 출력 : PDF 있음 크롤링 함수 시작
cr_pdf 출력 : 0번째 PDF 크롤링 중
[]
cr_pdf 출력 : 1번째 PDF 크롤링 중
[' 또한 이 집합투자증권은  “예금자보호법”에 의한 보호를  받지않는  실적배당상품으로  투자원금의  손실이  발생할  수 있으므로  투자에  신중을  기하여  주시기  바랍니다 .', ' 집합투자증권은  집합투자기구의  운용실적에  따라 손익이  결정되는  실적배당상품으로  예금자보호법에 따라 예금보험공사가  보호하지  아니하며 , 특히 예금자보호법의  적용을  받는 은행 등에서  집합투자증권을  매입하는  경우에도  은행예금과  달리 예금자보호법 에 따라 예금보험공사가  보호하지  않습니다 .', ' 집합투자증권은  ｢예금자보호법 ｣에 따라 예금보험공사가  보호하지  않는 실적배당상품이며 , 투자신탁  자산총액 의 60% 이상을 KOSPI200 지수, S&P500 지수, EUROSTOXX50 지수의 가격변동에 연계된  파생결합증권 에 투자함으로써  기초자산  가격의 변동에  따라 수익을  얻는 것을 목적으로  합니다 .', ' 투자자  유의사항  \uf06c 집합투자증권은  [예금자보호법 ]에 따라 예금보험공사가  보호하지  않는 실적배당상품으로  투자원금의  손실이  발생할  수 있으므로  투자에  신중을  기하여  주시기  바랍니다 .', ' 또한, 이 투자신탁은  「예금자보호법」의  적용을  받는 은행의  예금과  달리 실적에  따른 수익을  취득하므로  은행 등에서  판매하는  경우에도  「예금자보호법」에  따라 예금보험공사가  보호하지  않습니다 .']
 또한 이 집합투자증권은  “예금자보호법”에 의

NameError: name 'protect_img_url' is not defined